## Rewrite Retrieve Read is a method for Query rewriting for Retrieval augmented Large language models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0)

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
simple_query = "what is langchain?"

In [ ]:
chain.invoke(simple_query)

In [ ]:
distracted_query = "man that sam bankman fried trial was crazy! what is langchain?"

In [ ]:
chain.invoke(distracted_query)

In [ ]:
retriever(distracted_query)

## Rewrite-Retrieve-Read Implementation
- The main part is a rewriter to rewrite the search query

In [ ]:
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")

In [ ]:
print(rewrite_prompt.template)

In [ ]:
# Parser to remove the `**`


def _parse(text):
    return text.strip("**")

In [ ]:
rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse

In [ ]:
rewriter.invoke({"x": distracted_query})

In [ ]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rewrite_retrieve_read_chain.invoke(distracted_query)